In [1]:
import sys
sys.path.append("../")

In [2]:
import pandas as pd
import datetime as dt
from technicals.indicators import moving_average_convergence_divergence
from simulation.strategy_tester import StrategyTester

In [3]:
BUY = 1
SELL = -1
NONE = 0

def apply_signal(row):
    if row.direction == BUY and row.mid_l > row.EMA_100:
        return BUY
    if row.direction == SELL and row.mid_h < row.EMA_100:
        return SELL
    return NONE   

In [4]:
pair = "EUR_USD"
df_an = pd.read_pickle(f"../data/forex/{pair}_H1.pkl")
df_m5 = pd.read_pickle(f"../data/forex/{pair}_M5.pkl")
df_an.reset_index(drop=True, inplace=True)
df_m5.reset_index(drop=True, inplace=True)

In [5]:
df_an = moving_average_convergence_divergence(df_an)
df_an['EMA_100'] = df_an.mid_c.ewm(span=100, min_periods=100).mean()
df_an.dropna(inplace=True)
df_an.reset_index(drop=True, inplace=True)

In [6]:
df_an.head()

,time,volume,mid_o,mid_h,mid_l,mid_c,bid_o,bid_h,bid_l,bid_c,ask_o,ask_h,ask_l,ask_c,moving_average_convergence_divergence,signal,histogram,EMA_100
0,2010-01-07 20:00:00+00:00,1650,1.43270,1.43270,1.43074,1.43228,1.43265,1.43265,1.43069,1.43223,1.43274,1.43274,1.43078,1.43232,-0.001815,-0.001521,-0.000294,1.437059
1,2010-01-07 21:00:00+00:00,792,1.43230,1.43236,1.43079,1.43079,1.43225,1.43232,1.43069,1.43069,1.43234,1.43244,1.43089,1.43089,-0.001878,-0.001592,-0.000285,1.436916
2,2010-01-07 22:00:00+00:00,878,1.43089,1.43150,1.43046,1.43054,1.43079,1.43141,1.43037,1.43045,1.43099,1.43159,1.43055,1.43063,-0.001925,-0.001659,-0.000266,1.436771
3,2010-01-07 23:00:00+00:00,511,1.43057,1.43182,1.43050,1.43175,1.43048,1.43175,1.43042,1.43168,1.43066,1.43189,1.43057,1.43182,-0.001844,-0.001696,-0.000148,1.436657
4,2010-01-08 00:00:00+00:00,1337,1.43178,1.43273,1.43025,1.43171,1.43171,1.43266,1.43018,1.43164,1.43185,1.43280,1.43032,1.43178,-0.001763,-0.001709,-0.000053,1.436545


In [7]:
def apply_cross(row):
    if row.MACD_DELTA > 0 and row.MACD_DELTA_PREV < 0:
        return BUY
    if row.MACD_DELTA < 0 and row.MACD_DELTA_PREV > 0:
        return SELL
    return NONE

In [10]:
df_an['MACD_DELTA'] = df_an.moving_average_convergence_divergence - df_an.signal
df_an['MACD_DELTA_PREV'] = df_an.MACD_DELTA.shift(1)
df_an['direction'] = df_an.apply(apply_cross, axis=1)

In [13]:
df_an.direction.value_counts()

 0    70208
 1     2938
-1     2937
Name: direction, dtype: int64

In [4]:
def run_pair(pair):
    df_an = pd.read_pickle(f"../data/forex/{pair}_H1.pkl")
    df_m5 = pd.read_pickle(f"../data/forex/{pair}_M5.pkl")
    df_an.reset_index(drop=True, inplace=True)
    df_m5.reset_index(drop=True, inplace=True)
    df_an = moving_average_convergence_divergence(df_an)

    df_an['EMA_100'] = df_an.mid_c.ewm(span=100, min_periods=100).mean()
    our_cols = ['time', 'mid_o', 'mid_h', 'mid_l', 'mid_c',
            'bid_o', 'bid_h', 'bid_l', 'bid_c', 
            'ask_o', 'ask_h', 'ask_l', 'ask_c',
            'ENGULFING', 'direction', 'EMA_200', 'relative_strength_index_14' ]
    df_slim = df_an[our_cols].copy()
    df_slim.dropna(inplace=True)
    
    df_slim.reset_index(drop=True, inplace=True)
    gt = StrategyTester(
        df_slim,
        apply_signal,
        df_m5,
        use_spread=True
    )
    
    gt.run_test()
    return gt.df_results
    

In [5]:
res = []
for p in ["AUD_NZD", "EUR_USD", "GBP_JPY", "USD_CAD" ]:
    res.append(dict(pair=p,res=run_pair(p)))

prepare_data...
run_test...
Result: -4523.5
prepare_data...
run_test...
Result: -1828.0
prepare_data...
run_test...
Result: -2326.5
prepare_data...
run_test...
Result: -2652.5


In [6]:
for r in res:
    print(r['pair'], r['res'].result.sum())

AUD_NZD -4523.5
EUR_USD -1828.0
GBP_JPY -2326.5
USD_CAD -2652.5
